#4 MODEL IO

#4.1 FewShotPromptTemplate - Give a example template

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate


chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

chain = prompt | chat

chain.invoke({"country": "France"})

#4.2 FewShotChatMessagePromptTemplate

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate


chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know about {country}?"),
        ("ai", "{answer}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert, you give short answers."),
        example_prompt,
        ("human", "What do you know about {country}?"),
    ]
)

chain = final_prompt | chat

chain.invoke({"country": "Korea"})

#4.3 LengthBasedExampleSelector

LengthBaseExampleSelector-  Limit the Lenght of Answer

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.few_shot import FewShotPromptTemplate


chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

example_selector =  LengthBasedExampleSelector(
    examples=examples,  
    example_prompt=example_prompt,
    max_length=300,  
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector = example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

chain = prompt | chat
#prompt.format(country="Korea")
chain.invoke({"country": "Japan"})

BaseExampleSelector - Your own example selector

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.example_selector.base import BaseExampleSelector
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts import example_selector
from langchain.prompts.few_shot import FewShotPromptTemplate


chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice
        return [choice(self.examples)]

example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

example_selector = RandomExampleSelector(
    examples=examples,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

prompt.format(country="France")

#4.4 Serialization and Composition

load and save prompt from disk(json, yaml)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt

                     
chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

#prompt =  load_prompt("/mnt/c/Users/akoge/Documents/projects/source/gpt/fullstack-gpt/prompts/prompt.json")
#prompt =  load_prompt("../prompts/prompt.json")
prompt =  load_prompt("../prompts/prompt.yaml")

prompt.format(country="Germany")

PipelinePromptTemplate - many prompt put together

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate


chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)
example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)
start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)
final_prompt = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)

pipeline_prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]
prompt = PipelinePromptTemplate(
    final_prompt=final_prompt,
    pipeline_prompts=pipeline_prompts,
)

prompt.format(
    character= "Pirate",
    example_question= "What is your location?",
    example_answer= "Arrrrg! That is a secret!! Arg arg!!",
    question= "What is your fav food?",   
)

#chain = prompt | chat

# chain.invoke(
#     {
#         "character": "Pirate",
#         "example_question": "What is your location?",
#         "example_answer": "Arrrrg! That is a secret!! Arg arg!!",
#         "question": "What is your fav food?",
#     }
# )

#4.5 Caching

in Memory Cache and Debug

In Memeory cache only program is running. computer를 다시 재시작할 경우 케시는 지워진다.

In [ ]:
from langchain.cache import InMemoryCache
from langchain.globals import set_llm_cache, set_debug
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler


set_llm_cache(InMemoryCache())
set_debug(True)

chat = ChatOpenAI(
    temperature=0.1,
    # streaming=True,
    # callbacks=[
    # StreamingStdOutCallbackHandler(),
    # ],
)

chat.predict("How do you make italian pasta")

Run Same Questions and got from cache

In [ ]:
chat.predict("How do you make italian pasta")

Database Cache - can save on database

https://python.langchain.com/docs/integrations/providers/

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.cache import SQLiteCache
from langchain.globals import set_llm_cache, set_debug


set_llm_cache(SQLiteCache("../cache/cache.db"))

chat = ChatOpenAI(
    temperature=0.1,
    # streaming=True,
    # callbacks=[
    #     StreamingStdOutCallbackHandler(),
    # ],
)

chat.predict("How do you make italian pasta")

Run Same Questions and got from cache

In [ ]:
chat.predict("How do you make italian pasta")

https://python.langchain.com/docs/integrations/llms/llm_caching

#4.6 Serialization

Usages

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback


chat = ChatOpenAI(
    temperature=0.1,
)

with get_openai_callback() as usage:
    a = chat.predict("What is the recipe for soju")
    b = chat.predict("What is the recipe for bread")
    print(a, "\n")
    print(b, "\n")
    print(usage)
    #print(usage.total_cost)

Save setting to model.json

In [27]:
from langchain.llms.openai import OpenAI


chat = OpenAI(
    model="gpt-3.5-turbo-16k",
    temperature=0.1,
    max_tokens=450,
)

chat.save("../models/model.json")


Load model

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.llms.loading import load_llm


chat = load_llm("../models/model.json")
chat